# OpenLostCat Initialization and Query

In [7]:
import folium
from ipywidgets import interact

import numpy as np
import pandas as pd
import time

## pip install openlostcat
from openlostcat.main_osm_categorizer import MainOsmCategorizer
from openlostcat.osmqueryutils.ask_osm import ask_osm, ask_osm_around_point_df

Overpass query for hotels in Budapest: 

In [8]:
budapest_req = """
[out:json];
    (area["name"="Budapest"];) -> .searchArea;
    nwr[tourism=hotel](area.searchArea);
out tags center;
"""

Query the hotels:

In [9]:
budapest_hotels = ask_osm(budapest_req)
len(budapest_hotels['elements'])

260

Extract the main attributes and organize hotel data into a data frame:

In [11]:
[nwr.update(nwr['center']) for nwr in budapest_hotels['elements'] if 'center' in nwr]


bp_hotels = pd.DataFrame([[nwr['id'], 
                           nwr['lat'], nwr['lon'], 
                           nwr['tags'].get('name', 'NoName'),  
                           nwr['tags']] for nwr in budapest_hotels['elements']], 
                         columns = ['id', 'lat', 'lng', 'name', 'tags'])

Query map objects in the proximity of each hotel - repeatedly in case of incompleteness (failure): 

In [12]:
osm = bp_hotels[["lat", "lng"]].apply(lambda x: ask_osm_around_point_df(x, distance = 300), axis = 1)
bp_hotels["osm"] = osm

In [13]:
while len(bp_hotels.loc[bp_hotels.osm.isna(), "osm"]) > 0:
    time.sleep(5)
    osm_isna = bp_hotels[bp_hotels.osm.isna()][["lat", "lng"]].apply(lambda x: ask_osm_around_point_df(x, distance = 300), axis = 1)
    bp_hotels.loc[bp_hotels.osm.isna(), "osm"] = osm_isna

KeyboardInterrupt: 

Define our visualization function:

In [10]:
# If a category is empty, the map is positioned at the centroid of all hotels:
zero_location = np.array(list(zip(bp_hotels["lat"],bp_hotels["lng"]))).mean(axis=0)

def show_geo(related_geo, color_map, get_color_func):
    tmp = related_geo
    # extract coordinates
    coords = np.array(list(zip(tmp["lat"],tmp["lng"])))
    # extract other resources
#     Some coding issue: https://github.com/python-visualization/folium/issues/1320
    names = list([str(name.encode('raw_unicode_escape'))[2:-1] for name in tmp.name])
    tags = list([str(str(tag).encode('raw_unicode_escape'))[2:-1] for tag in tmp.tags])
    colors = get_color_func(tmp, color_map)
    m = folium.Map(
        location=coords.mean(axis=0) if len(coords) > 0 else zero_location,
        zoom_start=12,
        tiles='Stamen Terrain'
    )
    for i, loc in enumerate(coords):
        folium.Marker(loc, popup='<i>%s</i>' % tags[i], tooltip=str(names[i]), icon=folium.Icon(color=colors[i])).add_to(m)
    display(m)

# Example Scenarios

## Public Transport Accessibility

> Categorization in this example is based on the degree of public transport availability in the proximity of each hotel:
> * _Primary public transport accessibility_ means a fast and frequent rail connection is available nearby (light rail, subway or train service).
> * _Standard public transport accessibility_ means any other public transport service is available nearby.
> * _Non-accessibility_ means no public transport service is available nearby.

Initialize the categorizer and print categories:

In [11]:
categorizer = MainOsmCategorizer('rules/publictransport_rules.json')
print(categorizer.get_categories_enumerated_key_map())
print(categorizer)

{0: 'pt_primary_accessible', 1: 'pt_accessible', 2: 'pt_nonaccessible'}
CategoryCatalog:
category rule collection: [
    Category name: pt_primary_accessible
    rules: [
        ANY[8768385819829](
            and(
                {public_transport : {'stop_position'}}, is_optional_key = False
                or[
                    {light_rail : {'yes'}}, is_optional_key = False
                    {subway : {'yes'}}, is_optional_key = False
                    {train : {'yes'}}, is_optional_key = False
                ]
            )
        )
    ]
    Category name: pt_accessible
    rules: [
        OR[
            ANY[8768385824377](
                {public_transport : {'stop_position'}}, is_optional_key = False
            )
            ANY[8768385824437](
                {public_transport : {'platform'}}, is_optional_key = False
            )
            ANY[8768385824395](
                {amenity : {'ferry_terminal'}}, is_optional_key = False
            )
        ]
    ]
  

Do the categorization and get the assigned category indices:

In [ ]:
bp_hotels["pt_cat"] = [i[0] for i in bp_hotels.osm.map(categorizer.categorize)]

Count the number of hotels in each category:

In [ ]:
bp_hotels.pt_cat.map(categorizer.get_categories_enumerated_key_map()).value_counts()

Visualize on a map:

In [13]:
color_map_pt = dict(zip([0, 1, 2], ["green", "orange", "red"]))
get_color_func_pt = lambda df, color_map: list(df["pt_cat"].apply(lambda x: color_map.get(x, "black")))
key_map_pt = dict(zip(['pt_primary_accessible', 'pt_accessible', 'pt_nonaccessible'], [0, 1, 2]))



transport = ['All', 'pt_primary_accessible', 'pt_accessible', 'pt_nonaccessible']
@interact(pt_cat=transport)
def get_transport(pt_cat):
    selected = bp_hotels[bp_hotels.pt_cat == key_map_pt[pt_cat]] if pt_cat != 'All' else bp_hotels
    show_geo(selected, color_map_pt, get_color_func_pt)

interactive(children=(Dropdown(description='pt_cat', options=('All', 'pt_primary_accessible', 'pt_accessible',…

## Wheelchair-Accessible (Barrier-Free) Shopping

> Categorization in this example is based on the wheelchair-accessibility of supermarkets in the proximity of each hotel:
> * _No wheelchair shopping_ means there is no barrier-free supermarket nearby.
> * _Wheelchair-shopping paradise_ means every nearby supermarket is wheelchair-friendly.
> * _Wheelchair-shopping_ means there is at least one wheelchair-accessible supermarket.

Initialize the categorizer and print categories:

In [20]:
categorizer1W = MainOsmCategorizer('rules/wheelchair.json')
print(categorizer1W.get_categories_enumerated_key_map())
print(categorizer1W)

{0: 'no_wheelchair_shopping', 1: 'wheelchair_shopping_paradise', 2: 'wheelchair_shopping'}
CategoryCatalog:
category rule collection: [
    Category name: no_wheelchair_shopping
    rules: [
        ALL[8768270967507](
            impl(
                {shop : {'supermarket'}}, is_optional_key = False
                 => 
                {wheelchair : {'no'}}, is_optional_key = True
            )
        )
    ]
    Category name: wheelchair_shopping_paradise
    rules: [
        ALL[8768385824344](
            impl(
                {shop : {'supermarket'}}, is_optional_key = False
                 => 
                {wheelchair : {'limited', 'designated', 'yes'}}, is_optional_key = False
            )
        )
    ]
    Category name: wheelchair_shopping
    rules: [
        CONST(True)
    ]
]


Do the categorization and get the assigned category indices:

In [ ]:
bp_hotels["wheelchair"] = [i[0] for i in bp_hotels.osm.map(categorizer1W.categorize)]

Count the number of hotels in each category:

In [21]:
bp_hotels.wheelchair.value_counts()

0    143
2     71
1     45
Name: wheelchair, dtype: int64

Visualize on a map:

In [22]:
color_map_wc = dict(zip([1, 2, 0], ["green", "blue", "black"]))
get_color_func_wc = lambda df, color_map: list(df["wheelchair"].apply(lambda x: color_map.get(x, "black")))
key_map_wc = dict(zip(['wheelchair_shopping_paradise', 'wheelchair_shopping', 'no_wheelchair_shopping'], [1, 2, 0]))

wc = ['All', 'wheelchair_shopping_paradise', 'wheelchair_shopping', 'no_wheelchair_shopping']
@interact(wc_cat=wc)
def get_wc(wc_cat):
    selected = bp_hotels[bp_hotels.wheelchair == key_map_wc[wc_cat]] if wc_cat != 'All' else bp_hotels
    show_geo(selected, color_map_wc, get_color_func_wc)

interactive(children=(Dropdown(description='wc_cat', options=('All', 'wheelchair_shopping_paradise', 'wheelcha…

## Attractive Neighborhood: Water Proximity and Calm Streets

> Categorization in this example is based on whether the river bank is nearby and wheter all streets in the proximity of the hotel are at most tertiary graded. These aspects are evaluated in parallel and the best locations are in both categories:
> * _Water nearby_ means the river bank is near the hotel,
> * _Calm streets_ means there is no primary or secondary road nearby.

Initialize the categorizer and print categories (note: we are using the all-matching strategy here):

In [14]:
categorizer2 = MainOsmCategorizer('rules/attractive_neighborhood.json')
print(categorizer2.get_categories_enumerated_key_map())
print(categorizer2)

{0: 'water_nearby', 1: 'calm_streets'}
CategoryCatalog:
category rule collection: [
    Category name: water_nearby
    rules: [
        ANY[8768270955078](
            {waterway : {'river'}}, is_optional_key = False
        )
    ]
    Category name: calm_streets
    rules: [
        ALL[not_road](
            not(
                {highway : {'primary', 'secondary'}}, is_optional_key = False
            )
        )
    ]
]


Do the categorization and get the assigned category combinations:

In [ ]:
nb_cat = [[cat[0] for cat in cat_list] for cat_list in bp_hotels.osm.map(categorizer2.categorize)]
bp_hotels["water_nearby"] = [0 in cats  for cats in nb_cat]
bp_hotels["calm_streets"] = [1 in cats  for cats in nb_cat]

Count the number of hotels in each category combination:

In [15]:
print("calm_streets and water: " + str(bp_hotels[bp_hotels.water_nearby & bp_hotels.calm_streets].shape[0]))
print("only water: " + str(bp_hotels[(bp_hotels.water_nearby) & (bp_hotels.calm_streets == False)].shape[0]))
print("only calm_streets: " + str(bp_hotels[(bp_hotels.water_nearby == False) & (bp_hotels.calm_streets)].shape[0]))
print("nothing: " + str(bp_hotels[(bp_hotels.water_nearby == False) & (bp_hotels.calm_streets == False)].shape[0]))

calm_streets and water: 7
only water: 20
only calm_streets: 40
nothing: 192


Visualize on a map:

In [16]:
color_map_nb = {
    (True, False): "blue",
    (False, True): "green",
    (True, True): "purple",
    (False, False): "black"
}
get_color_func_nb = lambda df, color_map: list(map(lambda x: color_map.get(x, "black"), list(zip(df.water_nearby, df.calm_streets))))
nearby = ['All', 'water_nearby', 'calm_streets', 'both', 'none']
@interact(nearby_cat=nearby)
def get_nerby(nearby_cat):
    water_nearby = lambda x: x[x.water_nearby]
    calm_streets = lambda x: x[x.calm_streets]
    both = lambda x: x[x.water_nearby & x.calm_streets]
    none = lambda x: x[(x.water_nearby == False) & (x.calm_streets == False)]
    switch = {
        'water_nearby': water_nearby,
        'calm_streets': calm_streets,
        'both': both,
        'none': none
    }
    selected = switch.get(nearby_cat, lambda x: x)(bp_hotels)
    show_geo(selected, color_map_nb, get_color_func_nb)

interactive(children=(Dropdown(description='nearby_cat', options=('All', 'water_nearby', 'calm_streets', 'both…

## Preferred Location Based on the Combination of Public Transport Availability and Attractive Neighborhood

> Categorization in this example is based on the mixture of the public transport availability and the attractive neighborhood examples above. Note: The rule collection file defines references for each sub-category generated by these aspects.
> * _1st-priority preferred location_ means a hotel with primary public transport availability and either water closeness or calm streets in the neighborhood.
> * _2nd-priority preferred location_ means general public transport service availability and _both_ water closeness and calm streets.
> * _3rd-priority preferred location_ means either water closness or calm streets without any reference to public transport access.
> * _Not preferred location_ means the neighborhood is not attractive in terms of water proximity or calm streets.

Initialize the categorizer and print categories:

In [17]:
categorizer3 = MainOsmCategorizer('rules/mix.json')
print(categorizer3.get_categories_enumerated_key_map())
print(categorizer3)

{0: '1st_preferred_location', 1: '2nd_preferred_location', 2: '3rd_preferred_location', 3: 'not_preferred_location'}
CategoryCatalog:
category rule collection: [
    Category name: 1st_preferred_location
    rules: [
        AND(
            REF ##pt_primary_accessible(
                ANY[8768271237629](
                    and(
                        {public_transport : {'stop_position'}}, is_optional_key = False
                        or[
                            {light_rail : {'yes'}}, is_optional_key = False
                            {subway : {'yes'}}, is_optional_key = False
                            {train : {'yes'}}, is_optional_key = False
                        ]
                    )
                )
            )
            REF ##water_OR_calm_streets(
                OR[
                    REF ##water_nearby(
                        ANY[8768271237464](
                            {waterway : {'river'}}, is_optional_key = False
                        )
      

Do the categorization and get the assigned category indices:

In [ ]:
bp_hotels["preferred_cat"] = [i[0] for i in bp_hotels.osm.map(categorizer3.categorize)]

Count the number of hotels in each category:

In [18]:
bp_hotels.preferred_cat.value_counts()

3    192
2     49
0     12
1      6
Name: preferred_cat, dtype: int64

Visualize on a map:

In [19]:
color_map_mix = dict(zip([0, 1, 2, 3], ["green", "blue", "orange", "black"]))
get_color_func_mix = lambda df, color_map: list(df["preferred_cat"].apply(lambda x: color_map.get(x, "black")))
key_map_mix = dict(zip(['1st_preferred_location', '2nd_preferred_location', '3rd_preferred_location', 'not_preferred_location'], [0, 1, 2, 3]))

preffered = ['All', '1st_preferred_location', '2nd_preferred_location', '3rd_preferred_location', 'not_preferred_location']
@interact(preffered_cat=preffered)
def get_nerby(preffered_cat):
    selected = bp_hotels[bp_hotels.preferred_cat == key_map_mix[preffered_cat]] if preffered_cat != 'All' else bp_hotels
    show_geo(selected, color_map_mix, get_color_func_mix)

interactive(children=(Dropdown(description='preffered_cat', options=('All', '1st_preferred_location', '2nd_pre…